In [1]:
import numpy as np
from pycomposer.composition_controler import CompositionControler
from pycomposer.inferablepitch.rtrbm_inferer import RTRBMInferer
from pycomposer.inferableconsonance.q_consonance import QConsonance
from pycomposer.inferableduration.time_diff_inferer import TimeDiffInferer
from pycomposer.midi_vectorlizer import MidiVectorlizer

In [2]:
midi_vectorlizer = MidiVectorlizer()
tone_df = midi_vectorlizer.extract("../../furudokei.mid")
tone_df = tone_df.sort_values(by=["start", "end"])

In [3]:
total_measure_n = 100
measure_n = 16
beat_n = 9
metronome_time = 300

In [4]:
r_dict = {
    (1, 1): 5.0,
    (15, 16): 0.0,
    (8, 9): 0.0,
    (5, 6): 0.5,
    (4, 5): 0.5,
    (2, 1): 5.0,
    (32, 45): 0.0,
    (2, 3): 5.0,
    (5, 8): 0.5,
    (3, 5): 0.5,
    (9, 16): 0.0,
    (8, 15): 0.0,
    (1, 2): 5.0
}

inferable_consonance = QConsonance()
inferable_consonance.epsilon_greedy_rate = 0.9
inferable_consonance.alpha_value = 0.6
inferable_consonance.gamma_value = 0.7
inferable_consonance.initialize(r_dict=r_dict, tone_df=tone_df)
for pitch in tone_df.pitch.values.tolist():
    inferable_consonance.learn(state_key=pitch, limit=1)

In [5]:
import matplotlib.pyplot as plt
q_df = inferable_consonance.q_df.copy()
q_df["rate"] = q_df["state_key"] / q_df["action_key"]
#q_df = q_df[q_df.q_value > 0]
q_df.plot.scatter(x="rate", y="q_value")
plt.show();plt.close()
q_df.sort_values(by=["q_value"], ascending=False).head()

,state_key,action_key,q_value,rate
0,62.0,62.0,1.000000,1.000000
0,71.0,66.0,0.875770,1.075758
0,43.0,40.0,0.583449,1.075000
0,59.0,59.0,0.431973,1.000000
0,47.0,57.0,0.412037,0.824561


In [6]:
inferable_pitch = RTRBMInferer(
    learning_rate=0.00001,
    hidden_n=100,
    hidden_binary_flag=True,
    inferancing_training_count=1,
    r_batch_size=200
)

In [7]:
tone_df = tone_df[tone_df.program == 22]
inferable_pitch.learn(tone_df=tone_df, training_count=1, batch_size=200)

In [8]:
inferable_duration = TimeDiffInferer(
    tone_df,
    learning_rate=0.1,
    beat_n=beat_n,
    hidden_n=100,
    hidden_binary_flag=False,
    inferancing_training_count=1,
    r_batch_size=200
)

In [9]:
inferable_duration.learn(
    training_count=1, 
    batch_size=200,
    metronome_time=metronome_time,
    measure_n=measure_n,
    beat_n=beat_n,
    total_measure_n=400
)

In [10]:
composition_controler = CompositionControler(
    resolution=960,
    initial_tempo=120
)

In [11]:
chord_list = composition_controler.create_chord_list(
    octave=5, 
    first_chord="IV",
    total_measure_n=total_measure_n
)

composition_controler.compose_chord(
    chord_list,
    metronome_time=metronome_time, 
    start_measure_n=0,
    measure_n=measure_n,
    beat_n=beat_n,
    chord_instrument_num=34,
    chord_velocity_range=(90, 95)
)

In [12]:
composition_controler.compose_melody(
    inferable_pitch,
    inferable_consonance,
    inferable_duration,
    chord_list,
    total_measure_n=total_measure_n,
    measure_n=measure_n,
    start_measure_n=0,
    beat_n=beat_n,
    metronome_time=metronome_time,
    melody_instrument_num=0,
    melody_velocity_range=(120, 127)
)

In [13]:
composition_controler.save("midi/test10.mid")

In [14]:
df = midi_vectorlizer.extract("midi/test10.mid")

In [15]:
df

,program,start,end,pitch,velocity,duration
303,0,0.000000,0.310417,65,125,0.310417
304,0,0.310417,0.802083,59,123,0.491667
305,0,0.802083,1.473958,64,121,0.671875
306,0,1.473958,2.288021,67,124,0.814063
307,0,2.288021,2.615625,72,124,0.327604
308,0,2.615625,2.869792,64,123,0.254167
309,0,2.869792,3.000521,67,126,0.130729
310,0,3.000521,3.501562,72,126,0.501042
311,0,3.501562,4.092187,65,126,0.590625
312,0,4.092187,5.446875,65,122,1.354688
